In [1]:
import time as t
import datetime as dt
import pandas as pd

In [2]:
telegram_api_id = '2560748'
telegram_api_hash = 'f5ac2f0ae5f1374ee38f80e7c553f53f'
telegram_phone_number = '+8562099257429'
pass2fa = 'password'
from telethon import TelegramClient, events

In [3]:
client = TelegramClient(session=None,
                        api_id=telegram_api_id,
                        auto_reconnect=True,
                        api_hash=telegram_api_hash)

In [4]:
await client.start(phone=telegram_phone_number, password=pass2fa)

Attempt 1 at connecting failed: TimeoutError: 


Signed in successfully as Jack Ma; remember to not break the ToS or you will risk an account ban!


In [5]:
async for dialog in client.iter_dialogs():
    print(dialog.name, 'has ID', dialog.id)

Telegram has ID 777000
Solana New Liquidity Pools has ID -1002039712427
Solana Scanner has ID -1002023951506
Gambles 🎲 MadApes has ID -1001758611100
Degen Seals has ID -1001523523939
Marat Gudkov has ID 178008364
Jack Ma has ID 1223950100
Yield Board has ID -1001628892345


In [6]:
lp_chat = -1002039712427

In [41]:

## Rewrite to read last message
message =  await client.get_messages(lp_chat, limit=1)
message = pd.DataFrame({'messages':message})
message['id'] = message['messages'].apply(lambda x:x.id)
message['date'] = message['messages'].apply(lambda x:x.date)
message['text'] = message['messages'].apply(lambda x:x.text)
message['date'] = pd.to_datetime(message['date'])
message['date'] = message['date'].dt.tz_localize(None)

pattern = r'`([1-9A-HJ-NP-Za-km-z]{32,44})`'
message['address'] = message['text'].str.extract(pattern)
message['ltime'] = message['text'].str.extract(r'(?<=Launch:\*\* )(.*)')
message['ltime2'] = message['text'].str.extract(r'(?<=Trading Start Time:\*\* )(.*)')
message['Mcap'] = message['text'].str.extract(r'(?<=Mcap:\*\* )(.*)')
message['Mcap2'] = message['text'].str.extract(r'(?<=Marketcap:\*\* )(.*)')
message['Liq'] = message['text'].str.extract(r'(?<=Liq:\*\* )(.*)')
message['Liq2'] = message['text'].str.extract(r'(?<=Liquidity:\*\* )(.*)')
message['s_mm'] = message['text'].str.extract(r'(?<=Mutable Metadata: )(.*)')
message['s_ma'] = message['text'].str.extract(r'(?<=Mint Authority: )(.*)')
message['s_fa'] = message['text'].str.extract(r'(?<=Freeze Authority: )(.*)')
message['s_s'] = message['text'].str.extract(r'(?<=Score: )(.*)')

In [42]:
message['ltime'] = message['ltime'].combine_first(message['ltime2']).str.strip('`')
message['Mcap'] = message['Mcap'].combine_first(message['Mcap2']).str.strip('`')
message['Liq'] = message['Liq'].combine_first(message['Liq2']).str.strip('`')
message.loc[message['s_mm'].str.contains('Yes'), 's_mm2'] = True
message.loc[message['s_mm'].str.contains('No'), 's_mm2'] = False
message.loc[message['s_ma'].str.contains('Yes'), 's_ma2'] = True
message.loc[message['s_ma'].str.contains('No'), 's_ma2'] = False
message.loc[message['s_fa'].str.contains('Yes'), 's_fa2'] = True
message.loc[message['s_fa'].str.contains('No'), 's_fa2'] = False
message.loc[message['s_s'].str.contains('Bad'), 's_q'] = 0
message.loc[message['s_s'].str.contains('Neutral'), 's_q'] = 1
message.loc[message['s_s'].str.contains('Good'), 's_q'] = 2
message['s_sni'] = message['s_s'].str.extract(r'([0-9]+)')
message['s_sni'] = message['s_sni'].fillna(0)

mult = {'K':1e3,'M':1e6,'B':1e9,'T':1e12}
message['Mcap'] = message['Mcap'].str.strip('[$\*]')
message['Mcap_num'] = message['Mcap'].str.extract('([0-9\.]+)').astype(float) * message['Mcap'].str.extract('([A-Z])').replace(mult).fillna(1)
message['Liq'] = message['Liq'].str.strip(r" \[.*\]")
message['Liq_num'] = message['Liq'].str.extract('([0-9\.]+)').astype(float) * message['Liq'].str.extract('([A-Z])').replace(mult).fillna(1)
message = message.drop(columns=['ltime2','Mcap2','Liq2','s_mm','s_ma','s_fa','s_s','Mcap','Liq','text'])
message[['s_q','Mcap_num','Liq_num']] = message[['s_q','Mcap_num','Liq_num']].astype(int)

In [43]:
##write this to SQLite
message

,messages,id,date,address,ltime,s_mm2,s_ma2,s_fa2,s_q,s_sni,Mcap_num,Liq_num
0,"Message(id=215028, peer_id=PeerChannel(channel...",215028,2024-04-18 01:00:20,8EkFW8K4cqYmtxzqL3m5n44hzF1QMUwjACkH3Go57N11,9 seconds ago,False,False,False,2,0,723,1320
